# AdaBoost (Adaptive Boosting) Complete Implementation Guide

## Overview
AdaBoost is a powerful ensemble learning algorithm that combines multiple weak learners (typically decision stumps) to create a strong classifier. This notebook provides comprehensive coverage of AdaBoost for both classification and regression tasks, with detailed hyperparameter optimization using GridSearchCV.

## Learning Objectives
- Master AdaBoost algorithm principles and implementation
- Understand comprehensive hyperparameter tuning with GridSearchCV
- Compare baseline vs optimized model performance
- Analyze parameter impact on model performance
- Implement complete prediction pipelines for deployment

## Key Concepts
- **Adaptive Boosting**: Sequential learning focusing on misclassified examples
- **Weak Learners**: Simple models (decision stumps) combined into strong ensemble
- **Weight Adjustment**: Dynamic sample weight updates based on prediction errors
- **GridSearchCV**: Systematic hyperparameter optimization with cross-validation

## Technical Stack
- **scikit-learn**: AdaBoost implementation and optimization tools
- **GridSearchCV**: Comprehensive hyperparameter search
- **Performance Analysis**: Multiple evaluation metrics and visualizations

In [ ]:
# Import Essential Libraries for Comprehensive AdaBoost Implementation
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_classification, make_regression
from sklearn.model_selection import train_test_split, GridSearchCV, validation_curve
from sklearn.ensemble import AdaBoostClassifier, AdaBoostRegressor
from sklearn.metrics import (accuracy_score, classification_report, confusion_matrix,
                           mean_squared_error, mean_absolute_error, r2_score)
import warnings
warnings.filterwarnings('ignore')

# Set visualization style for professional plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ All libraries imported successfully")
print("📊 Visualization style configured")
print("🚀 Ready for comprehensive AdaBoost implementation")

## 1. Dataset Creation and Exploration

We'll create both classification and regression datasets to demonstrate AdaBoost's versatility across different problem types.

In [ ]:
# Create Classification Dataset
X_clf, y_clf = make_classification(
    n_samples=1000,          # Total samples for robust training/testing
    n_features=20,           # Feature dimensionality
    n_informative=15,        # Informative features for classification
    n_redundant=3,           # Redundant features (linear combinations)
    n_clusters_per_class=1,  # Cluster structure per class
    random_state=42          # Reproducible results
)

# Create Regression Dataset
X_reg, y_reg = make_regression(
    n_samples=1000,          # Total samples
    n_features=15,           # Feature dimensionality
    n_informative=12,        # Informative features
    noise=0.1,               # Noise level
    random_state=42          # Reproducible results
)

# Split datasets
X_clf_train, X_clf_test, y_clf_train, y_clf_test = train_test_split(
    X_clf, y_clf, test_size=0.2, random_state=42, stratify=y_clf
)

X_reg_train, X_reg_test, y_reg_train, y_reg_test = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=42
)

print("📊 Dataset Summary:")
print("=" * 50)
print(f"Classification Dataset:")
print(f"  • Training samples: {X_clf_train.shape[0]}")
print(f"  • Testing samples: {X_clf_test.shape[0]}")
print(f"  • Features: {X_clf_train.shape[1]}")
print(f"  • Class distribution: {dict(zip(*np.unique(y_clf, return_counts=True)))}")

print(f"\nRegression Dataset:")
print(f"  • Training samples: {X_reg_train.shape[0]}")
print(f"  • Testing samples: {X_reg_test.shape[0]}")
print(f"  • Features: {X_reg_train.shape[1]}")
print(f"  • Target range: [{y_reg.min():.2f}, {y_reg.max():.2f}]")

## 2. GridSearchCV Setup and Configuration

GridSearchCV provides systematic hyperparameter optimization through exhaustive search over parameter combinations with cross-validation. Understanding its configuration is crucial for effective model tuning.

In [ ]:
# Configure GridSearchCV Parameters for Comprehensive Search

def create_gridsearch_config(estimator, param_grid, task_type='classification'):
    """
    Create standardized GridSearchCV configuration with optimal settings.
    
    Parameters:
    -----------
    estimator : sklearn estimator
        The base estimator to optimize
    param_grid : dict
        Parameter grid for systematic search
    task_type : str
        'classification' or 'regression' for appropriate scoring
    
    Returns:
    --------
    GridSearchCV object with optimized configuration
    """
    
    # Select appropriate scoring metric
    scoring = 'accuracy' if task_type == 'classification' else 'neg_mean_squared_error'
    
    # Configure GridSearchCV with comprehensive settings
    grid_search = GridSearchCV(
        estimator=estimator,        # Base model to optimize
        param_grid=param_grid,      # Parameter combinations to test
        cv=5,                       # 5-fold cross-validation for robust evaluation
        scoring=scoring,            # Optimization metric
        n_jobs=-1,                  # Use all available CPU cores
        verbose=3,                  # Detailed progress reporting
        return_train_score=True,    # Return training scores for analysis
        refit=True                  # Refit best model on full training set
    )
    
    return grid_search

print("⚙️ GridSearchCV Configuration Function Created")
print("🔧 Key Configuration Parameters:")
print("   • Cross-validation: 5-fold for robust performance estimation")
print("   • Parallel processing: All available CPU cores (-1)")
print("   • Verbose output: Detailed progress monitoring (level 3)")
print("   • Training scores: Included for overfitting analysis")
print("   • Automatic refit: Best model fitted on complete training data")

## 3. Parameter Grid Definition and Detailed Explanation

Understanding each AdaBoost parameter's impact is crucial for effective hyperparameter tuning. Let's define comprehensive parameter grids with detailed explanations.

In [ ]:
# Define Comprehensive Parameter Grids with Detailed Explanations

# AdaBoost Classification Parameter Grid
param_grid_classification = {
    'n_estimators': [50, 100, 200],           # Number of boosting stages
    'learning_rate': [0.01, 0.1, 1.0, 1.5, 2.0],  # Shrinkage parameter
    'algorithm': ['SAMME', 'SAMME.R']         # Boosting algorithm variant
}

# AdaBoost Regression Parameter Grid  
param_grid_regression = {
    'n_estimators': [50, 100, 200],           # Number of boosting stages
    'learning_rate': [0.01, 0.1, 1.0, 1.5, 2.0],  # Shrinkage parameter
    'loss': ['linear', 'square', 'exponential']     # Loss function for regression
}

print("🎯 AdaBoost Parameter Grid Analysis")
print("=" * 60)
print("\n📋 CLASSIFICATION PARAMETERS:")
print("━" * 40)

print("\n🔢 n_estimators (Number of Boosting Stages):")
print("   • 50: Fast training, may underfit")
print("   • 100: Balanced performance/speed")
print("   • 200: Better accuracy, slower training")
print("   💡 Impact: More estimators = better performance but higher overfitting risk")

print("\n📈 learning_rate (Shrinkage Parameter):")
print("   • 0.01: Very conservative, requires many estimators")
print("   • 0.1: Conservative, good generalization")  
print("   • 1.0: Default, no shrinkage")
print("   • 1.5: Aggressive learning")
print("   • 2.0: Very aggressive, high overfitting risk")
print("   💡 Impact: Lower rates = better generalization, higher rates = faster convergence")

print("\n🔄 algorithm (Boosting Algorithm Variant):")
print("   • SAMME: Discrete AdaBoost, uses class predictions")
print("   • SAMME.R: Real AdaBoost, uses class probabilities (default)")
print("   💡 Impact: SAMME.R typically faster and more accurate")

print("\n📋 REGRESSION PARAMETERS:")
print("━" * 40)

print("\n📉 loss (Loss Function for Regression):")
print("   • linear: Linear loss, less sensitive to outliers")
print("   • square: Squared loss, standard for regression")
print("   • exponential: Exponential loss, very sensitive to outliers")
print("   💡 Impact: Choice affects sensitivity to outliers and convergence behavior")

# Calculate total parameter combinations
clf_combinations = np.prod([len(values) for values in param_grid_classification.values()])
reg_combinations = np.prod([len(values) for values in param_grid_regression.values()])

print(f"\n🔍 Search Space Analysis:")
print(f"   • Classification combinations: {clf_combinations}")
print(f"   • Regression combinations: {reg_combinations}")
print(f"   • Total evaluations with 5-fold CV: {(clf_combinations + reg_combinations) * 5}")

## 4. Comprehensive Hyperparameter Tuning Execution

Now we'll execute the systematic hyperparameter search for both classification and regression tasks, monitoring the optimization progress.

In [ ]:
# Execute Comprehensive AdaBoost Classification Optimization

print("🚀 Starting AdaBoost Classification Hyperparameter Optimization")
print("=" * 65)

# Initialize base AdaBoost classifier
ada_classifier = AdaBoostClassifier(random_state=42)

# Create GridSearchCV for classification
grid_search_clf = create_gridsearch_config(
    estimator=ada_classifier,
    param_grid=param_grid_classification,
    task_type='classification'
)

print("⏳ Executing comprehensive parameter search...")
print("   This process evaluates all parameter combinations with 5-fold CV")
print("   Progress will be displayed below...\n")

# Execute the grid search
grid_search_clf.fit(X_clf_train, y_clf_train)

print(f"\n✅ Classification Optimization Complete!")
print(f"🏆 Best Parameters: {grid_search_clf.best_params_}")
print(f"📊 Best CV Accuracy: {grid_search_clf.best_score_:.4f}")
print(f"⚡ Total fits performed: {len(grid_search_clf.cv_results_)}")

# Store classification results for later analysis
best_clf_model = grid_search_clf.best_estimator_
best_clf_params = grid_search_clf.best_params_
best_clf_score = grid_search_clf.best_score_

In [ ]:
# Execute Comprehensive AdaBoost Regression Optimization

print("\n🚀 Starting AdaBoost Regression Hyperparameter Optimization")
print("=" * 62)

# Initialize base AdaBoost regressor
ada_regressor = AdaBoostRegressor(random_state=42)

# Create GridSearchCV for regression
grid_search_reg = create_gridsearch_config(
    estimator=ada_regressor,
    param_grid=param_grid_regression,
    task_type='regression'
)

print("⏳ Executing comprehensive parameter search...")
print("   Optimizing for negative mean squared error")
print("   Progress will be displayed below...\n")

# Execute the grid search
grid_search_reg.fit(X_reg_train, y_reg_train)

print(f"\n✅ Regression Optimization Complete!")
print(f"🏆 Best Parameters: {grid_search_reg.best_params_}")
print(f"📊 Best CV Score (neg_MSE): {grid_search_reg.best_score_:.4f}")
print(f"📈 Best CV RMSE: {np.sqrt(-grid_search_reg.best_score_):.4f}")
print(f"⚡ Total fits performed: {len(grid_search_reg.cv_results_)}")

# Store regression results for later analysis  
best_reg_model = grid_search_reg.best_estimator_
best_reg_params = grid_search_reg.best_params_
best_reg_score = grid_search_reg.best_score_

## 5. Best Model Extraction and Validation

With optimization complete, let's extract the best models and validate their performance on test sets.

In [ ]:
# Validate Best Classification Model Performance

print("🎯 Best Classification Model Validation")
print("=" * 45)

# Make predictions with optimized classifier
y_clf_pred_optimized = best_clf_model.predict(X_clf_test)
clf_test_accuracy = accuracy_score(y_clf_test, y_clf_pred_optimized)

# Create baseline for comparison
baseline_clf = AdaBoostClassifier(random_state=42)
baseline_clf.fit(X_clf_train, y_clf_train)
y_clf_pred_baseline = baseline_clf.predict(X_clf_test)
clf_baseline_accuracy = accuracy_score(y_clf_test, y_clf_pred_baseline)

print(f"📊 Classification Performance Summary:")
print(f"   • Best Parameters: {best_clf_params}")
print(f"   • Cross-validation Accuracy: {best_clf_score:.4f}")
print(f"   • Test Set Accuracy: {clf_test_accuracy:.4f}")
print(f"   • Baseline Accuracy: {clf_baseline_accuracy:.4f}")
print(f"   • Improvement: {clf_test_accuracy - clf_baseline_accuracy:.4f}")
print(f"   • Relative Improvement: {((clf_test_accuracy - clf_baseline_accuracy) / clf_baseline_accuracy * 100):.2f}%")

print(f"\n🔍 Model Architecture:")
print(f"   • Number of Estimators: {len(best_clf_model.estimators_)}")
print(f"   • Feature Importances Available: {len(best_clf_model.feature_importances_)}")
print(f"   • Estimator Weights Shape: {best_clf_model.estimator_weights_.shape}")

# Detailed classification report
print(f"\n📋 Detailed Classification Report:")
print(classification_report(y_clf_test, y_clf_pred_optimized, target_names=['Class 0', 'Class 1']))

In [ ]:
# Validate Best Regression Model Performance

print("\n🎯 Best Regression Model Validation")
print("=" * 42)

# Make predictions with optimized regressor
y_reg_pred_optimized = best_reg_model.predict(X_reg_test)

# Create baseline for comparison
baseline_reg = AdaBoostRegressor(random_state=42)
baseline_reg.fit(X_reg_train, y_reg_train)
y_reg_pred_baseline = baseline_reg.predict(X_reg_test)

# Calculate comprehensive regression metrics
reg_mse_optimized = mean_squared_error(y_reg_test, y_reg_pred_optimized)
reg_rmse_optimized = np.sqrt(reg_mse_optimized)
reg_mae_optimized = mean_absolute_error(y_reg_test, y_reg_pred_optimized) 
reg_r2_optimized = r2_score(y_reg_test, y_reg_pred_optimized)

reg_mse_baseline = mean_squared_error(y_reg_test, y_reg_pred_baseline)
reg_rmse_baseline = np.sqrt(reg_mse_baseline)
reg_r2_baseline = r2_score(y_reg_test, y_reg_pred_baseline)

print(f"📊 Regression Performance Summary:")
print(f"   • Best Parameters: {best_reg_params}")
print(f"   • Cross-validation Score (neg_MSE): {best_reg_score:.4f}")
print(f"   • Cross-validation RMSE: {np.sqrt(-best_reg_score):.4f}")

print(f"\n📈 Test Set Performance (Optimized vs Baseline):")
print(f"   • MSE:  {reg_mse_optimized:.4f} vs {reg_mse_baseline:.4f}")
print(f"   • RMSE: {reg_rmse_optimized:.4f} vs {reg_rmse_baseline:.4f}")
print(f"   • MAE:  {reg_mae_optimized:.4f} vs {reg_mae_baseline:.4f}")
print(f"   • R²:   {reg_r2_optimized:.4f} vs {reg_r2_baseline:.4f}")

print(f"\n💡 Performance Improvements:")
print(f"   • RMSE Reduction: {reg_rmse_baseline - reg_rmse_optimized:.4f}")
print(f"   • R² Improvement: {reg_r2_optimized - reg_r2_baseline:.4f}")
print(f"   • Relative RMSE Improvement: {((reg_rmse_baseline - reg_rmse_optimized) / reg_rmse_baseline * 100):.2f}%")

print(f"\n🔍 Model Architecture:")
print(f"   • Number of Estimators: {len(best_reg_model.estimators_)}")
print(f"   • Feature Importances Available: {len(best_reg_model.feature_importances_)}")
print(f"   • Estimator Weights Shape: {best_reg_model.estimator_weights_.shape}")

## 6. Parameter Impact Visualization

Understanding how different parameters affect model performance helps in making informed tuning decisions. Let's create comprehensive visualizations.

In [ ]:
# Create Parameter Performance Heatmaps

def create_parameter_heatmap(grid_search, param1, param2, title):
    """Create heatmap showing parameter interaction effects."""
    
    # Extract results from grid search
    results_df = pd.DataFrame(grid_search.cv_results_)
    
    # Create pivot table for heatmap
    pivot_table = results_df.pivot_table(
        values='mean_test_score',
        index=f'param_{param1}',
        columns=f'param_{param2}',
        aggfunc='mean'
    )
    
    # Create heatmap
    plt.figure(figsize=(10, 6))
    sns.heatmap(pivot_table, annot=True, fmt='.4f', cmap='viridis', cbar_kws={'label': 'CV Score'})
    plt.title(f'{title}\nParameter Interaction Heatmap')
    plt.xlabel(param2.replace('_', ' ').title())
    plt.ylabel(param1.replace('_', ' ').title())
    plt.tight_layout()
    plt.show()

# Import pandas for data manipulation
import pandas as pd

print("📊 Parameter Impact Visualization Analysis")
print("=" * 50)

# Classification parameter heatmap
create_parameter_heatmap(
    grid_search_clf, 
    'n_estimators', 
    'learning_rate',
    'AdaBoost Classification'
)

# Algorithm comparison for classification
results_clf = pd.DataFrame(grid_search_clf.cv_results_)
algorithm_performance = results_clf.groupby('param_algorithm')['mean_test_score'].agg(['mean', 'std'])

print(f"\n🔍 Algorithm Performance Comparison (Classification):")
print(algorithm_performance.round(4))

plt.figure(figsize=(8, 5))
algorithm_performance['mean'].plot(kind='bar', yerr=algorithm_performance['std'], capsize=4)
plt.title('Algorithm Performance Comparison - Classification')
plt.ylabel('Cross-Validation Accuracy')
plt.xlabel('Algorithm')
plt.xticks(rotation=0)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Regression Parameter Analysis and Validation Curves

print("\n📈 Regression Parameter Analysis")
print("=" * 40)

# Regression parameter heatmap
create_parameter_heatmap(
    grid_search_reg,
    'n_estimators', 
    'learning_rate',
    'AdaBoost Regression'
)

# Loss function comparison for regression
results_reg = pd.DataFrame(grid_search_reg.cv_results_)
loss_performance = results_reg.groupby('param_loss')['mean_test_score'].agg(['mean', 'std'])

print(f"\n🔍 Loss Function Performance Comparison (Regression):")
print("Note: Scores are negative MSE (higher is better)")
print(loss_performance.round(4))

plt.figure(figsize=(8, 5))
loss_performance['mean'].plot(kind='bar', yerr=loss_performance['std'], capsize=4)
plt.title('Loss Function Performance Comparison - Regression')
plt.ylabel('Cross-Validation Score (neg_MSE)')
plt.xlabel('Loss Function')
plt.xticks(rotation=0)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Create validation curves for key parameters
print("\n📊 Generating Validation Curves...")

# Validation curve for n_estimators (classification)
estimator_range = [10, 25, 50, 75, 100, 150, 200]
train_scores, test_scores = validation_curve(
    AdaBoostClassifier(random_state=42), 
    X_clf_train, y_clf_train,
    param_name='n_estimators',
    param_range=estimator_range,
    cv=5, scoring='accuracy', n_jobs=-1
)

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(estimator_range, train_scores.mean(axis=1), 'o-', label='Training Score')
plt.plot(estimator_range, test_scores.mean(axis=1), 'o-', label='Validation Score')
plt.fill_between(estimator_range, train_scores.mean(axis=1) - train_scores.std(axis=1),
                 train_scores.mean(axis=1) + train_scores.std(axis=1), alpha=0.2)
plt.fill_between(estimator_range, test_scores.mean(axis=1) - test_scores.std(axis=1),
                 test_scores.mean(axis=1) + test_scores.std(axis=1), alpha=0.2)
plt.xlabel('Number of Estimators')
plt.ylabel('Accuracy Score')
plt.title('Validation Curve - n_estimators (Classification)')
plt.legend()
plt.grid(True, alpha=0.3)

# Validation curve for learning_rate (classification)
lr_range = [0.01, 0.1, 0.5, 1.0, 1.5, 2.0]
train_scores_lr, test_scores_lr = validation_curve(
    AdaBoostClassifier(random_state=42),
    X_clf_train, y_clf_train,
    param_name='learning_rate',
    param_range=lr_range,
    cv=5, scoring='accuracy', n_jobs=-1
)

plt.subplot(1, 2, 2)
plt.plot(lr_range, train_scores_lr.mean(axis=1), 'o-', label='Training Score')
plt.plot(lr_range, test_scores_lr.mean(axis=1), 'o-', label='Validation Score')
plt.fill_between(lr_range, train_scores_lr.mean(axis=1) - train_scores_lr.std(axis=1),
                 train_scores_lr.mean(axis=1) + train_scores_lr.std(axis=1), alpha=0.2)
plt.fill_between(lr_range, test_scores_lr.mean(axis=1) - test_scores_lr.std(axis=1),
                 test_scores_lr.mean(axis=1) + test_scores_lr.std(axis=1), alpha=0.2)
plt.xlabel('Learning Rate')
plt.ylabel('Accuracy Score')
plt.title('Validation Curve - learning_rate (Classification)')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✅ Parameter impact analysis complete!")

## 7. Model Deployment and Prediction Pipeline

Finally, let's create a complete prediction pipeline using our optimized models for real-world deployment scenarios.

In [ ]:
# Create Production-Ready Prediction Pipeline

class AdaBoostPredictionPipeline:
    """
    Production-ready AdaBoost prediction pipeline with comprehensive functionality.
    """
    
    def __init__(self, classification_model=None, regression_model=None):
        self.classification_model = classification_model
        self.regression_model = regression_model
        self.model_info = {}
        
        # Store model information
        if classification_model:
            self.model_info['classification'] = {
                'parameters': classification_model.get_params(),
                'n_estimators': len(classification_model.estimators_),
                'feature_importances': classification_model.feature_importances_
            }
            
        if regression_model:
            self.model_info['regression'] = {
                'parameters': regression_model.get_params(),
                'n_estimators': len(regression_model.estimators_),
                'feature_importances': regression_model.feature_importances_
            }
    
    def predict_classification(self, X, return_probabilities=False):
        """Make classification predictions with optional probability scores."""
        if self.classification_model is None:
            raise ValueError("No classification model available")
            
        predictions = self.classification_model.predict(X)
        
        if return_probabilities:
            probabilities = self.classification_model.predict_proba(X)
            return predictions, probabilities
        
        return predictions
    
    def predict_regression(self, X):
        """Make regression predictions."""
        if self.regression_model is None:
            raise ValueError("No regression model available")
            
        return self.regression_model.predict(X)
    
    def get_feature_importance(self, task_type='classification'):
        """Get feature importance scores."""
        if task_type == 'classification' and self.classification_model:
            return self.classification_model.feature_importances_
        elif task_type == 'regression' and self.regression_model:
            return self.regression_model.feature_importances_
        else:
            raise ValueError(f"No {task_type} model available")
    
    def model_summary(self):
        """Display comprehensive model summary."""
        print("🚀 AdaBoost Prediction Pipeline Summary")
        print("=" * 50)
        
        for task_type, info in self.model_info.items():
            print(f"\n📊 {task_type.title()} Model:")
            print(f"   • Parameters: {info['parameters']}")
            print(f"   • Number of Estimators: {info['n_estimators']}")
            print(f"   • Feature Importance Available: {len(info['feature_importances'])} features")

# Initialize the production pipeline
pipeline = AdaBoostPredictionPipeline(
    classification_model=best_clf_model,
    regression_model=best_reg_model
)

# Display pipeline summary
pipeline.model_summary()

print("\n✅ Production pipeline initialized successfully!")
print("🔧 Available methods:")
print("   • predict_classification(X, return_probabilities=False)")
print("   • predict_regression(X)")
print("   • get_feature_importance(task_type)")
print("   • model_summary()")

In [ ]:
# Demonstrate Pipeline Usage with Example Predictions

print("🧪 Pipeline Demonstration with Example Predictions")
print("=" * 55)

# Generate sample data for demonstration
n_samples = 5
sample_clf_data = X_clf_test[:n_samples]
sample_reg_data = X_reg_test[:n_samples]

print(f"📝 Making predictions on {n_samples} sample observations...\n")

# Classification predictions
print("🎯 Classification Predictions:")
print("-" * 30)
clf_predictions = pipeline.predict_classification(sample_clf_data)
clf_predictions_proba = pipeline.predict_classification(sample_clf_data, return_probabilities=True)

for i in range(n_samples):
    actual = y_clf_test[i]
    predicted = clf_predictions[i]
    prob_class_0 = clf_predictions_proba[1][i][0]
    prob_class_1 = clf_predictions_proba[1][i][1]
    
    print(f"Sample {i+1}: Actual={actual}, Predicted={predicted}")
    print(f"         Probabilities: Class 0={prob_class_0:.3f}, Class 1={prob_class_1:.3f}")

# Regression predictions
print(f"\n📈 Regression Predictions:")
print("-" * 25)
reg_predictions = pipeline.predict_regression(sample_reg_data)

for i in range(n_samples):
    actual = y_reg_test[i]
    predicted = reg_predictions[i]
    error = abs(actual - predicted)
    
    print(f"Sample {i+1}: Actual={actual:.3f}, Predicted={predicted:.3f}, Error={error:.3f}")

# Feature importance analysis
print(f"\n🔍 Feature Importance Analysis:")
print("-" * 35)

clf_importance = pipeline.get_feature_importance('classification')
reg_importance = pipeline.get_feature_importance('regression')

print(f"Classification - Top 5 Important Features:")
top_clf_features = np.argsort(clf_importance)[-5:][::-1]
for i, feature_idx in enumerate(top_clf_features):
    print(f"  {i+1}. Feature {feature_idx}: {clf_importance[feature_idx]:.4f}")

print(f"\nRegression - Top 5 Important Features:")
top_reg_features = np.argsort(reg_importance)[-5:][::-1]
for i, feature_idx in enumerate(top_reg_features):
    print(f"  {i+1}. Feature {feature_idx}: {reg_importance[feature_idx]:.4f}")

print(f"\n✅ Pipeline demonstration complete!")
print(f"🚀 Models are ready for production deployment")

## Summary and Key Insights

This comprehensive AdaBoost implementation demonstrates advanced hyperparameter optimization techniques and production-ready deployment strategies.

### 🎯 Key Findings:

**GridSearchCV Optimization:**
- Systematic parameter search significantly improves model performance
- Cross-validation provides robust performance estimates
- Parameter interactions reveal optimal configurations

**Parameter Impact Analysis:**
- **n_estimators**: More estimators generally improve performance but increase overfitting risk
- **learning_rate**: Lower rates provide better generalization, higher rates faster convergence
- **algorithm/loss**: SAMME.R typically outperforms SAMME for classification

**Production Deployment:**
- Complete prediction pipeline with error handling
- Feature importance analysis for model interpretability
- Comprehensive performance monitoring capabilities

### 🚀 Best Practices:

1. **Hyperparameter Tuning Strategy:**
   - Use GridSearchCV with cross-validation for robust optimization
   - Monitor both training and validation performance
   - Consider computational cost vs. performance trade-offs

2. **Model Selection:**
   - Start with reasonable parameter ranges
   - Use validation curves to understand parameter sensitivity
   - Balance model complexity with generalization ability

3. **Production Deployment:**
   - Implement comprehensive prediction pipelines
   - Include error handling and model monitoring
   - Maintain feature importance tracking for interpretability

### 🎓 Educational Value:
This notebook provides a complete workflow from basic implementation through advanced optimization to production deployment, demonstrating industry-standard practices for ensemble learning with AdaBoost.